# Overview

This Notebook demonstrates a minimal example of querying the cockroach database using the engine, session, and models defined in the backend code.  The backend code is being read-only volume mounted to `/home/jovyan/backend`, and that directory has been added to the `PYTHONPATH` environment variable.  Imports should "just work".

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

logging.basicConfig()
logger = logging.getLogger("sqlalchemy.engine")
logger.propagate = True
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

# Show backend engine

In [3]:
import sqlalchemy as sa
import rich

from app.db import engine, db_session
from app.models import UserDAO, TodoDAO

engine

<IPython.core.display.Javascript object>

In [4]:
db_session

<function app.db.db_session()>

<IPython.core.display.Javascript object>

# Query all Users

In [5]:
async with db_session() as session:
    statement = sa.select(UserDAO)
    results = await session.execute(statement)
    users = results.scalars().all()

users

INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:select version()
INFO:sqlalchemy.engine.Engine:[generated in 0.00093s] ()
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%s)
INFO:sqlalchemy.engine.Engine:[generated in 0.00086s] ('sqlalchemy-cockroachdb 1.4.3.dev0',)
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%s)
INFO:sqlalchemy.engine.Engine:[cached since 0.003728s ago] ('sqlalchemy 1.4',)
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT users.id, users.created_at, users.name, users.password 
FROM users
INFO:sqlalchemy.engine.Engine:[generated in 0.00104s] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


<IPython.core.display.Javascript object>

In [6]:
rich.print(users[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f4344c0d280>,
    'id': UUID('4d4cea9e-945c-455c-8f7a-66b279d89540'),
    'created_at': datetime.datetime(2021, 11, 29, 15, 14, 49, 707819, 
tzinfo=datetime.timezone.utc),
    'name': 'user2',
    'password': 'pass'
}

<IPython.core.display.Javascript object>

In [7]:
for u in users:
    print(u.name)

user2
user1
user3


<IPython.core.display.Javascript object>

# Query all Todos

In [8]:
async with db_session() as session:
    statement = sa.select(TodoDAO)
    results = await session.execute(statement)
    todos = results.scalars().all()
todos

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT todo.id, todo.created_at, todo.title, todo.content, todo.user_id, users_1.id AS id_1, users_1.created_at AS created_at_1, users_1.name, users_1.password 
FROM todo LEFT OUTER JOIN users AS users_1 ON users_1.id = todo.user_id
INFO:sqlalchemy.engine.Engine:[generated in 0.00093s] ()
INFO:sqlalchemy.engine.Engine:ROLLBACK


<IPython.core.display.Javascript object>

In [9]:
rich.print(todos[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f43449ef700>,
    'created_at': datetime.datetime(2021, 11, 29, 15, 14, 50, 269675, 
tzinfo=datetime.timezone.utc),
    'title': 'Note 2',
    'user_id': UUID('b32b67ab-af4b-438f-bd5f-74c162887037'),
    'content': 'Edit this Note',
    'id': UUID('78907ea7-0d8e-4821-bffe-2379b58c63cf'),
    'user': <app.models.UserDAO object at 0x7f43449ef7c0>
}

<IPython.core.display.Javascript object>

# Update a Todo

In [10]:
id = todos[0].id
id

UUID('78907ea7-0d8e-4821-bffe-2379b58c63cf')

<IPython.core.display.Javascript object>

In [11]:
from sqlalchemy.orm.attributes import set_attribute

async with db_session() as session:
    updates = {"title": "title42", "content": "modified by notebook"}
    statement = sa.select(TodoDAO).where(TodoDAO.id == id)
    results = await session.execute(statement)
    t = results.scalars().first()
    for k, v in updates.items():
        set_attribute(t, k, v)
    await session.commit()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT todo.id, todo.created_at, todo.title, todo.content, todo.user_id, users_1.id AS id_1, users_1.created_at AS created_at_1, users_1.name, users_1.password 
FROM todo LEFT OUTER JOIN users AS users_1 ON users_1.id = todo.user_id 
WHERE todo.id = %s
INFO:sqlalchemy.engine.Engine:[generated in 0.00068s] (UUID('78907ea7-0d8e-4821-bffe-2379b58c63cf'),)
INFO:sqlalchemy.engine.Engine:UPDATE todo SET title=%s, content=%s WHERE todo.id = %s
INFO:sqlalchemy.engine.Engine:[generated in 0.00146s] ('title42', 'modified by notebook', UUID('78907ea7-0d8e-4821-bffe-2379b58c63cf'))
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [12]:
t

<IPython.core.display.Javascript object>

In [13]:
t.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f4344999910>,
 'created_at': datetime.datetime(2021, 11, 29, 15, 14, 50, 269675, tzinfo=datetime.timezone.utc),
 'title': 'title42',
 'user_id': UUID('b32b67ab-af4b-438f-bd5f-74c162887037'),
 'content': 'modified by notebook',
 'id': UUID('78907ea7-0d8e-4821-bffe-2379b58c63cf'),
 'user': <app.models.UserDAO at 0x7f43449990a0>}

<IPython.core.display.Javascript object>